
Based on NLP From Scratch: Generating Names with a Character-Level RNN
*************************************************************
by: `Sean Robertson <https://github.com/spro/practical-pytorch>`_


In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string
import random
from random import shuffle
import torch
import torch.nn as nn
from functools import reduce
from collections import Counter
import re
import numpy as np
%matplotlib inline

In [2]:


class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size,num_layers=1, bias=True, batch_first=False,
                dropout=0, bidirectional=False):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        if bidirectional:
            self.num_directions = 2
        else:
            self.num_directions = 1
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, 
                            num_layers=num_layers, bias=bias, batch_first=batch_first,
                           dropout=dropout, bidirectional=bidirectional)
#         self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
#         self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)

        self.o2o = nn.Linear(self.num_directions * hidden_size, output_size)
#         self.dropout = nn.Dropout(0.1)
#         self.softmax = nn.Softmax(dim=2)

    def forward(self, my_input, hidden):
#         input_combined = torch.cat((category, input, hidden), 1)
#         hidden = self.i2h(input_combined)
#         output = self.i2o(input_combined)
#         output_combined = torch.cat((hidden, output), 1)
#         output = self.o2o(output_combined)
#         output = self.dropout(output)
        output, hidden = self.lstm(my_input, hidden)
        output = self.o2o(output)
#         output = self.softmax(output)
        return output, hidden

    def initHidden(self, batch=1, device=torch.device('cpu')):
        return (torch.zeros(self.num_layers,1,self.hidden_size, device=device), torch.zeros(self.num_layers * self.num_directions,batch,self.hidden_size,device=device))
#         return torch.zeros(1, self.hidden_size)

In [3]:

def string_to_chars(original):
    raw_lines = original.splitlines()

    def proc_line(l): return [l] if l == '<start>' or l == '<end>' else list(l)
    chars = reduce(lambda x, y : x + ['\n'] + y, list(map(proc_line, raw_lines)))
    assert original == ''.join(chars)
    return chars

def make_encoder(raw):
    chars = string_to_chars(raw)
    decoder = [c for c in Counter(chars)]
    decoder = sorted(decoder)
    return {c:i for i, c in enumerate(decoder)}, decoder

def split_songs(raw):
    return re.findall('(<start>.*?<end>)',raw,flags=re.DOTALL)

# train_songs = list(map(string_to_chars, split_songs(raw_train)))

# reconstructed = '\n'.join(map(''.join, train_songs))
# encoder, decoder = make_encoder(raw_train)
# n_chars = len(decoder)

def song_to_tensor_chunks(song,encoder, decoder):
    training_chunks = []
    target_chunks = []
    for i in range(0,len(song),100):
        start = i
        if start+100 > len(song) - 1:
            end = len(song)
            target_range = list(np.arange(start + 1, end,dtype=np.int)) + [int(end - 1)]
        else:
            end = start + 100
            target_range = np.arange(start+1, end+1, dtype=np.int)
        chunk = torch.zeros(end-start, 1, n_chars)
        for i, c in enumerate(song[start:end]):
            chunk[i,0,encoder[c]] = 1
        target_chunk = []
        for i in target_range:
            target_chunk.append(encoder[song[i]])
        target_chunks.append(torch.tensor(target_chunk))
        training_chunks.append(chunk)
    return training_chunks, target_chunks


In [4]:
raw_train = open('train.txt').read()
train_songs = list(map(string_to_chars, split_songs(raw_train)))

reconstructed = '\n'.join(map(''.join, train_songs))
encoder, decoder = make_encoder(raw_train)
n_chars = len(decoder)
val_songs = list(map(string_to_chars, split_songs(open('val.txt').read())))
test_songs = list(map(string_to_chars, split_songs(open('test.txt').read())))
assert reconstructed == raw_train


For convenience during training we'll make a ``randomTrainingExample``
function that fetches a random (category, line) pair and turns them into
the required (category, input, target) tensors.




Training the Network
--------------------

In contrast to classification, where only the last output is used, we
are making a prediction at every step, so we are calculating loss at
every step.

The magic of autograd allows you to simply sum these losses at each step
and call backward at the end.




In [5]:
criterion = nn.CrossEntropyLoss()

def train(song,optimizer=None, device=torch.device('cpu')):
    train_chunks, target_chunks = song_to_tensor_chunks(song, encoder,decoder)
    hidden = rnn.initHidden(device=device)
    rnn.zero_grad()

    loss = 0

    for input_line_tensor, target_line_tensor in zip(train_chunks,target_chunks):
        target_line_tensor.unsqueeze_(-1)
        for i in range(input_line_tensor.size(0)):
            output, hidden = rnn(input_line_tensor[i].view((1,1,n_chars)).to(device), hidden)
    #         print(output)
            l = criterion(output.view((1,n_chars)), target_line_tensor[i].to(device))
            loss += l / input_line_tensor.size(0)

    loss.backward()
    optimizer.step()

    return output, loss.item() / len(train_chunks)
def test(rnn, song, device=torch.device('cpu')):
    train_chunks, target_chunks = song_to_tensor_chunks(song, encoder,decoder)
    hidden = rnn.initHidden(device=device)
    loss = 0
    for input_line_tensor, target_line_tensor in zip(train_chunks,target_chunks):
        target_line_tensor.unsqueeze_(-1)
        output, hidden = rnn(input_line_tensor.to(device), hidden)
        seq_len,_ =target_line_tensor.size()
        l = criterion(output.view((seq_len,n_chars)), target_line_tensor.view((seq_len)).to(device))
        loss += l
    return output, loss.item() / len(train_chunks)

In [6]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [7]:
# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

CUDA is supported


In [8]:
rnn = RNN(n_chars, 128, n_chars).to(device)
name='rnn'
version_to_load = 29
rnn.load_state_dict(torch.load(f'{name}_{version_to_load}.pt'))
rnn = rnn.to(device)
train_loss = list(map(float,(open(f'train_loss_{version_to_load}.csv').read().splitlines())))
val_loss = list(map(float,(open(f'val_loss_{version_to_load}.csv').read().splitlines())))
print(len(train_loss))
start_epoch = len(train_loss)
optimizer = torch.optim.Adam(rnn.parameters())

# name='rnn_256_L2-9'
# rnn = RNN(n_chars, 256, n_chars).to(device)
# rnn = rnn.to(device)
# optimizer = torch.optim.Adam(rnn.parameters(), weight_decay=0.9)
# train_loss = []
# val_loss = []

# start_epoch = 0

30


In [9]:
# rnn = RNN(n_chars, 128, n_chars).to(device)
learning_rate = 0.05
# optimizer = torch.optim.Adam(rnn.parameters())
n_iters = 10000
print_every = 80
plot_every = 10
# train_loss = []
# val_loss = []
total_loss = 0 # Reset every plot_every iters
n_epochs = 20
start = time.time()

for epoch in range(int(start_epoch),int(start_epoch+n_epochs)):
    shuffle(train_songs)
    total_loss = 0
    for i, song in enumerate(train_songs):
        output, loss = train(song,optimizer=optimizer, device=device)
        total_loss += loss
        if i % print_every == 0:
            print('%s (%d %d%%) %.4f' % (timeSince(start), i, i / len(train_songs) * 100, loss))


    train_loss.append(total_loss / len(train_songs))
    with torch.no_grad():
        total_loss = 0
        for song in test_songs:
            _, loss = test(rnn, song, device=device)
            total_loss += loss
        val_loss.append(total_loss / len(test_songs))
    torch.save(rnn.state_dict(),f'{name}_{epoch}.pt')
    with open(f'{name}_train_loss_{epoch}.csv', 'w+') as out:
        out.write('\n'.join(map(str,train_loss)))

    with open(f'{name}_val_loss_{epoch}.csv', 'w+') as out:
        out.write('\n'.join(map(str,val_loss)))
    print('Epoch %d %s val loss: %.4f train loss: %.4f\n' % (epoch,timeSince(start), val_loss[-1], train_loss[-1]))   
        

0m 0s (0 0%) 4.5451
0m 29s (80 9%) 4.5032
0m 57s (160 19%) 4.4835
1m 26s (240 29%) 4.4520
2m 4s (320 39%) 4.4063
2m 35s (400 49%) 4.4190
3m 7s (480 59%) 4.4116
3m 38s (560 69%) 4.3981
4m 11s (640 79%) 4.3987
4m 43s (720 89%) 4.3817
5m 13s (800 99%) 4.3467
Epoch 0 5m 17s val loss: 4.3890 train loss: 4.4406

5m 18s (0 0%) 4.3560
5m 54s (80 9%) 4.3815
6m 28s (160 19%) 4.3891
7m 1s (240 29%) 4.4075
7m 33s (320 39%) 4.3889
8m 2s (400 49%) 4.3939
8m 33s (480 59%) 4.4429
9m 2s (560 69%) 4.3545
9m 32s (640 79%) 4.3774
10m 1s (720 89%) 4.3896
10m 30s (800 99%) 4.3838
Epoch 1 10m 33s val loss: 4.3741 train loss: 4.3899

10m 34s (0 0%) 4.4120
11m 6s (80 9%) 4.3724
11m 39s (160 19%) 4.4061
12m 11s (240 29%) 4.4014
12m 41s (320 39%) 4.4024
13m 17s (400 49%) 4.4342
13m 45s (480 59%) 4.3100
14m 19s (560 69%) 4.3084
14m 50s (640 79%) 4.3503
15m 19s (720 89%) 4.3881
15m 49s (800 99%) 4.4044
Epoch 2 15m 52s val loss: 4.3678 train loss: 4.3820

15m 53s (0 0%) 4.3334
16m 24s (80 9%) 4.4006
16m 52s (160 19

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
offset = 0
print(train_loss)
print(val_loss)
with open('train_loss_{offset}.csv', 'w+') as out:
    out.write('\n'.join(map(str,train_loss)))

with open('val_loss_{offset}.csv', 'w+') as out:
    out.write('\n'.join(map(str,val_loss)))
plt.figure()
plt.plot(train_loss)
plt.plot(val_loss)

In [9]:

def sample():
    with torch.no_grad():
        input_tensor_chunk, _ = song_to_tensor_chunks(['<start>'], encoder, decoder)
        input_tensor = input_tensor_chunk[0].to(device)
        hidden = rnn.initHidden(device=device)
        prev_char_raw, hidden =rnn(input_tensor,hidden) 
        prev_char_probs = torch.softmax(prev_char_raw, 2, prev_char_raw.dtype).view((n_chars))
        output = '<start>'
        while True:
            selection = np.random.random()
            running_prob = 0
            for i, prob in enumerate(prev_char_probs):
                running_prob += prob
                if running_prob > selection:
                    selected_char = decoder[i]
                    output += selected_char
                    input_tensor_chunk, _ = song_to_tensor_chunks([selected_char], encoder, decoder)
                    input_tensor = input_tensor_chunk[0].to(device)
                    break
            if selected_char == '<end>':
                break
            else:
                prev_char_raw, hidden =rnn(input_tensor,hidden) 
                prev_char_probs = torch.softmax(prev_char_raw, 2, prev_char_raw.dtype).view((n_chars))
        return output

print(sample())
# # Sample from a category and starting letter
# def sample(category, start_letter='A'):
#     with torch.no_grad():  # no need to track history in sampling
#         category_tensor = categoryTensor(category)
#         input = inputTensor(start_letter)
#         hidden = rnn.initHidden()

#         output_name = start_letter

#         for i in range(max_length):
#             output, hidden = rnn(category_tensor, input[0], hidden)
#             topv, topi = output.topk(1)
#             topi = topi[0][0]
#             if topi == n_letters - 1:
#                 break
#             else:
#                 letter = all_letters[topi]
#                 output_name += letter
#             input = inputTensor(letter)

#         return output_name

# # Get multiple samples from one category and multiple starting letters
# def samples(category, start_letters='ABC'):
#     for start_letter in start_letters:
#         print(sample(category, start_letter))

# samples('Russian', 'RUS')

# samples('German', 'GER')

# samples('Spanish', 'SPA')

# samples('Chinese', 'CHI')

<start>
XiM?4
T:Le sauro d?outene, Warnits
R:Bransle
C:Margs ce the Castain
D:Pardy Roge
C:Parrende Puphe Mazurka, The
L:1/8
Z:Transcrit et/ou corrig? par Michel BELLON - 2007-04-25
Z:Pour toute observation mailto:galouvielle@free.fr
M:8/4
L:1/4
L:1/8
K:C
|: Ad B2e2 | efe2 c2A2 | e2de e3/2G/2 | A6
:: frontte, La be n'ar-le pas re?cu pout tas-3-
g2-fe | d2 B2 | Bd c2 | 
Bd | A2 (cd | G4- | c3 Az | B2 dd B  | AB FE | EF AG | d3- BG | G4 | {a}d2(3fe | (3bwa ag |
c2 fd | ec c2 ||
c2) z2 :| c2A Bc | B2 Bd | d2 d2 | f3 ezfe | B3 B2c | B2 B2 B>B | c2 Az Bc | 
A3 AG | G>E DG | G3 G Bc | d2 dB AG | F2 A2 |
<end>


In [49]:
def heatmap_activations(seq_len):
    with torch.no_grad():
        input_tensor_chunk, _ = song_to_tensor_chunks(['<start>'], encoder, decoder)
        input_tensor = input_tensor_chunk[0].to(device)
        hidden = rnn.initHidden(device=device)
        h, _ = hidden
        activations = []
        output = ['<start>']
        prev_char_raw, hidden =rnn(input_tensor,hidden) 
        prev_char_probs = torch.softmax(prev_char_raw, 2, prev_char_raw.dtype).view((n_chars))
        for i in range(seq_len):
            selection = np.random.random()
            running_prob = 0
            for i, prob in enumerate(prev_char_probs):
                running_prob += prob
                if running_prob > selection:
                    selected_char = decoder[i]
                    input_tensor_chunk, _ = song_to_tensor_chunks([selected_char], encoder, decoder)
                    input_tensor = input_tensor_chunk[0].to(device)
                    break
            h_i, _ = hidden
            activations.append(h_i.squeeze())
            if selected_char == '<end>':
                selected_char = '<start>'
                input_tensor_chunk, _ = song_to_tensor_chunks(['<start>'], encoder, decoder)
                input_tensor = input_tensor_chunk[0].to(device)
                hidden = rnn.initHidden(device=device)
            output.append(selected_char)
            prev_char_raw, hidden =rnn(input_tensor,hidden)
            prev_char_probs = torch.softmax(prev_char_raw, 2, prev_char_raw.dtype).view((n_chars))
        return output[:-1], activations

In [115]:
def neural_activations(activations, i): return list(map(lambda x:x[i].item(), activations))
def make_heatmap_for_neuron(neuron, activations,inputs,shape=(20,30)):
    title = f"Heatmap of neuron: {neuron}"
    xlabel= ""
    ylabel=""
    data = np.array(neural_activations(activations,neuron)).reshape(shape)


    plt.figure(figsize=(20, 10))
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    c = plt.pcolor(data, edgecolors='k', linewidths=4, cmap='seismic', vmin=0.0, vmax=1.0)

    def show_values(pc, labels):
        pc.update_scalarmappable()
        ax = pc.axes
        print(pc.get_array())
        for p, color, value, label in zip(pc.get_paths(), pc.get_facecolors(), pc.get_array(),labels):
            x, y = p.vertices[:-2, :].mean(0)
            if np.all(color[:3] > 0.5):
                color = (0.0, 0.0, 0.0)
            else:
                color = (1.0, 1.0, 1.0)
            ax.text(x, y, label, ha="center", va="center", color=color)

    show_values(c,inputs)

    plt.colorbar(c)
    plt.savefig(f"heatmaps/Heatmap_{neuron}")
    plt.close()

In [116]:
import matplotlib.pyplot as plt

In [117]:
hidden_size = rnn.hidden_size
inputs, activations = heatmap_activations(600)
inputs = '<FOOBAR>'.join(inputs).replace('\n','\\n').replace(' ','sp').split('<FOOBAR>')

for i in range(hidden_size):
    make_heatmap_for_neuron(i,activations, inputs)

[ 0.39013314  0.35996461  0.1698041   0.20555279  0.79882133  0.77675372
  0.69743764 -0.67336148  0.46480352  0.62802851  0.40915823  0.83277392
  0.01485776  0.06768112  0.0144629   0.32530969  0.07366375  0.09166146
  0.68061888  0.72643548  0.17112447 -0.50094259  0.14885277  0.17576265
  0.40513688  0.74396574  0.16710036  0.4081175   0.65200424  0.75735486
  0.64449686  0.14270779  0.52251321 -0.52920628  0.62971503  0.42982656
  0.73167062  0.28908727  0.38196519  0.60371858  0.82381904  0.21460503
 -0.5784077   0.44682127  0.61930156  0.06955691  0.34566668  0.0032199
  0.13992691  0.74640846  0.11334559  0.77607805  0.01193194  0.8443622
  0.79070169  0.89451677  0.70725542 -0.60424596 -0.91563165 -0.63830948
  0.43112582  0.68875051  0.8183496   0.16835108  0.18735501  0.00722766
  0.07638898  0.29810268  0.85469347  0.86134815  0.31848761 -0.21044067
 -0.75171757  0.22382003  0.21397893  0.19787021  0.85341853 -0.52519935
 -0.84213227 -0.43588939  0.50108206 -0.43317497 -0.6

[-2.33911365e-01 -2.74049997e-01 -8.01336542e-02  6.90733567e-02
  2.24189103e-01  3.83894771e-01 -5.61762005e-02 -5.02802908e-01
 -2.14537755e-01 -5.39058089e-01 -2.96321124e-01 -6.42390728e-01
 -6.41707957e-01 -3.02198082e-01  1.65545300e-01  1.10752709e-01
 -5.88220954e-01  5.44146895e-01  4.97912318e-01 -5.63698053e-01
  3.05415839e-01  6.46422923e-01  8.14286292e-01  6.58019066e-01
  3.85486454e-01 -2.11621985e-01 -1.84874460e-01  3.26249599e-01
 -7.99922924e-03  3.95904958e-01  3.24059367e-01  4.31585431e-01
 -7.28474483e-02  5.94195724e-01  4.43999559e-01  1.73479676e-01
  3.29958141e-01  1.40412062e-01  4.30112898e-01  7.41290271e-01
  9.03087199e-01  4.24903452e-01 -3.45423460e-01  4.11836296e-01
 -1.63246140e-01 -3.88082922e-01 -5.76589227e-01  2.74044424e-01
  4.29633021e-01  7.61891365e-01  7.41109073e-01 -5.19888401e-01
 -3.69099349e-01  4.06449646e-01  9.12982151e-02 -7.47703761e-02
  5.31284392e-01 -1.16247520e-01  4.40307200e-01  6.28132045e-01
  8.21699262e-01  9.15767

[-0.52647924 -0.073498    0.23994137  0.57402068 -0.28960034 -0.17824799
 -0.10842203  0.13701937  0.4501344   0.77344197  0.24787913  0.30739406
  0.30566055  0.76435941  0.39332452  0.08164462  0.58046293  0.68845487
  0.38627926  0.1041364   0.49966305  0.51576895  0.73931533  0.53278387
  0.66564518  0.54079926  0.23209652  0.67336756  0.78473783  0.64603943
  0.55702603  0.72553575  0.44360605 -0.02211601  0.36023653  0.63986057
  0.55018634  0.42526677  0.32276338  0.26085696  0.75691724  0.67458594
 -0.08666655  0.12299626  0.06639861  0.07431597  0.31401104  0.61946583
  0.18467465  0.30843869  0.31252858  0.39104658  0.26026669  0.07387365
  0.72813594  0.36697674  0.05869279  0.50454015  0.38331109  0.5874905
  0.84485459  0.8901726   0.37178734  0.00835972  0.08920453  0.39271119
  0.08435289  0.12036885 -0.02465494  0.01334466  0.04550074  0.4063485
  0.791879    0.54730201  0.34888044  0.25829801  0.90466815  0.45816934
  0.8740052   0.98701322  0.87134004  0.86133409  0.7

[-6.05072856e-01 -4.08876203e-02 -7.79143453e-01 -1.92676540e-02
 -3.10666770e-01 -2.10305825e-01 -2.24120263e-02 -4.33915406e-01
 -4.69380058e-03 -6.64031744e-01 -1.80198371e-01 -1.29109398e-01
 -1.06565915e-01 -4.28412735e-01 -5.15410721e-01 -8.51659197e-03
 -8.46091509e-02 -3.34329270e-02 -5.56119502e-01 -6.09889448e-01
 -1.91430867e-01 -6.20481893e-02 -2.14758947e-01 -3.75022352e-01
 -1.17547475e-01 -1.49743473e-02 -1.47865221e-01 -1.66021530e-02
 -7.82330707e-02 -2.95639634e-01 -2.13479459e-01 -2.50527173e-01
 -2.45221499e-02 -8.99369568e-02 -2.52445750e-02 -5.33108056e-01
 -6.25171065e-01 -3.27968955e-01 -3.02703440e-01 -2.77535766e-01
 -1.69958219e-01 -2.46166717e-02 -2.24504903e-01 -3.67801785e-02
 -6.34757161e-01 -3.49510580e-01 -1.36855856e-01 -3.14667150e-02
 -7.33515382e-01 -5.93151331e-01 -5.62445641e-01 -1.14292972e-01
 -6.92994416e-01 -1.18281215e-01 -2.32502982e-01 -7.15992332e-01
 -1.45809978e-01 -2.79434174e-01 -3.27958129e-02 -4.91586141e-02
 -3.68248045e-01 -3.91421

[ 0.54914486  0.68717396  0.83313006  0.94078583  0.8015914   0.75279933
  0.61037439  0.75672448  0.91257137 -0.16240351 -0.33133402 -0.42525288
 -0.40385661 -0.38371468 -0.16209225 -0.54367274 -0.34829548 -0.19517352
  0.39206704 -0.34510991 -0.60067576 -0.71257371 -0.34799942 -0.6891917
 -0.74702793 -0.92987859 -0.94619703 -0.88744956 -0.73921812 -0.7435807
 -0.70057636 -0.81520236 -0.89940721  0.49366099  0.8099795   0.74528933
  0.48600912  0.00720643 -0.58279669 -0.69243968 -0.83248979 -0.93402427
 -0.15554363  0.62865591  0.71925318  0.66077918  0.42813057 -0.38529736
 -0.81852382 -0.74061912 -0.52858073 -0.66782957 -0.51994163 -0.78403527
 -0.82442474 -0.70674849 -0.8221975  -0.87483358 -0.77974313 -0.22356336
 -0.40160638 -0.60453272 -0.3284615  -0.81542975 -0.9719975  -0.94253337
 -0.98094338 -0.91724873 -0.16260599 -0.29731753 -0.248265   -0.3580693
  0.74210936 -0.06141105 -0.7099908  -0.90542072 -0.96998787 -0.92502999
 -0.97012937  0.49483585  0.56026596 -0.29352903 -0.66

...

[-2.66750485e-01 -1.04849122e-01 -7.09494650e-01  2.47308820e-01
  1.78622335e-01  1.53543755e-01  1.26671731e-01 -5.55998445e-01
 -5.12900472e-01 -6.33439541e-01 -9.79027599e-02  4.85286891e-01
  4.43037078e-02  2.26257697e-01  3.33412811e-02  8.61314237e-02
 -6.14519477e-01 -7.61007726e-01 -6.02886900e-02 -2.70433556e-02
 -4.25265133e-02 -1.80844739e-02 -2.01066793e-03  1.68527695e-04
  1.26632012e-03  2.09039636e-03  7.72515850e-05 -5.23377617e-04
 -1.12428917e-02  5.66591881e-03 -5.59321314e-04  2.11547571e-03
  4.98265540e-03 -7.06291556e-01 -8.22791636e-01 -7.10934520e-01
 -2.90686749e-02 -1.72722992e-02 -3.07900272e-02 -3.80225986e-01
  1.00678848e-02  5.72703220e-03 -5.16058862e-01 -4.49928313e-01
 -7.65929401e-01  5.19200146e-01  7.60600746e-01  5.59454322e-01
 -4.24682856e-01 -7.81866670e-01 -4.86629337e-01 -3.15969288e-02
  4.84840013e-04  6.51322782e-01  5.37082911e-01  6.38220981e-02
  3.73913556e-01  2.54253238e-01 -1.38654336e-01 -6.14375114e-01
 -5.51195920e-01  4.60731

[-0.33686605 -0.04408477  0.07118335  0.21454532 -0.08605161 -0.32834393
 -0.09410122  0.06159402  0.0407253   0.56310499  0.4533394   0.41091111
  0.71985269  0.09344664  0.45475033  0.1788359   0.13206625  0.01826673
  0.9340542   0.65593201  0.18871553  0.47422943  0.85961944  0.76709014
  0.89755648  0.36254355  0.44197312  0.5073362   0.20630647  0.55518854
  0.19684789  0.19022965 -0.02268654  0.00915631  0.64744377  0.91623646
  0.6129474   0.898938    0.75670713  0.855694    0.70382917  0.19719027
 -0.05605467  0.07869439  0.67240232  0.00874356  0.0355222   0.34397399
  0.37201518  0.72983557  0.65884775  0.73468715  0.95889467  0.78540373
  0.9420343   0.39949867  0.7551614   0.92548954  0.67909122  0.90949339
  0.99032182  0.9848389   0.97451693  0.7183373   0.97780579  0.98968142
  0.52762163  0.28148302  0.86920696  0.01781717  0.0033062   0.00448113
  0.34271264  0.25890854  0.91997349  0.93241155  0.8731516   0.67399985
  0.29143646 -0.53062576 -0.79078865 -0.46242961 -0

[-0.64251727  0.04552432 -0.03129873 -0.18837164 -0.67056102 -0.86192966
  0.25317842 -0.67954046  0.63453072 -0.50522304 -0.5650835  -0.56325227
  0.16513605 -0.58957595 -0.79946703  0.69016683 -0.66657507  0.5739336
 -0.72446227  0.09200676  0.3737798  -0.7344026   0.05446372 -0.74281847
 -0.81847119 -0.92961168 -0.62920344 -0.10642311 -0.76365525 -0.89638913
 -0.89495564 -0.96210587  0.54067177 -0.28516901  0.30084816 -0.75516093
  0.15665129 -0.62956983 -0.31101358 -0.80864441 -0.94601643  0.61006057
 -0.68808228 -0.56358874 -0.76032245 -0.75994855  0.27650908 -0.03835182
 -0.00574437 -0.48686221 -0.85725963 -0.34756219 -0.8762536   0.43973264
 -0.51958472 -0.91332537  0.0800788  -0.63159466 -0.46863842  0.71257275
 -0.60819077 -0.86248332 -0.88386428 -0.94234943 -0.65509087 -0.85373116
 -0.47699815  0.61215651 -0.33199847  0.45788389 -0.45709947  0.65159464
 -0.7442379  -0.65894532  0.03231864 -0.2618264   0.66179466 -0.72249985
  0.61642969 -0.71562582  0.33620706  0.65682065  0.

[-4.70561922e-01  4.14574683e-01  2.84053445e-01  7.01429963e-01
  6.24211311e-01  6.91648424e-01  8.57056379e-01  7.42408454e-01
  7.77876496e-01  4.88657981e-01  2.80905843e-01  4.97298092e-01
 -5.65417588e-01 -5.71869850e-01  1.39698312e-01  4.93963659e-01
  5.67595512e-02  5.11604965e-01  6.45823658e-01  6.21611536e-01
  8.57255161e-01  7.21059442e-01 -5.70894122e-01 -6.83871448e-01
  3.36635083e-01  7.27168545e-02  3.55646461e-02  2.91577011e-01
  4.29905832e-01  3.24703485e-01  7.57657230e-01  7.19001889e-01
  7.29654670e-01  8.18208575e-01  8.86091411e-01  2.32069984e-01
  6.30348980e-01 -7.04739034e-01 -8.64555180e-01 -7.52401710e-01
 -7.18862295e-01  4.73790407e-01  4.28503484e-01  8.90403688e-01
 -2.72563338e-01  7.60322586e-02 -1.52392730e-01 -7.32912183e-01
 -8.01601946e-01 -7.32172728e-01 -8.84115338e-01 -9.58673418e-01
 -9.83809233e-01 -9.57746983e-01 -8.94175172e-01 -2.44763851e-01
  5.33347800e-02  7.31784582e-01  2.98894584e-01  7.52860248e-01
 -3.92556518e-01  7.38359

[-0.50788677 -0.30511498  0.68839169 -0.39276871 -0.74610543 -0.93757015
 -0.91518027  0.648453    0.25856164 -0.31656516 -0.58497912 -0.71178764
 -0.71413207 -0.81371742 -0.86187881 -0.94127488  0.54669553  0.63965678
 -0.18596077 -0.68641913 -0.74199337 -0.82478678 -0.86151171 -0.96033406
 -0.7639727  -0.77794975 -0.86335623 -0.89259338 -0.56674582 -0.87151653
 -0.90732414 -0.9596253  -0.95459962  0.33783999 -0.37461463 -0.63125944
 -0.88137013 -0.81733435 -0.9897536  -0.95488334 -0.95926946 -0.97794998
  0.48877671 -0.25629053  0.45109329  0.85988355  0.74931097 -0.39791918
 -0.83610499 -0.794981    0.44974139 -0.68427515  0.1541397  -0.66013658
 -0.67810059  0.56041372  0.8672967   0.88936949  0.81191599  0.32352492
 -0.46193069 -0.34237236  0.46553072  0.81046945 -0.59659213  0.50056756
  0.63397622  0.12765688  0.47418019  0.77427793  0.94674683  0.84543848
  0.87303817 -0.30264014 -0.70017594 -0.84045011 -0.96095186 -0.98914868
 -0.99806345 -0.99414992 -0.98573905 -0.99652278 -0

In [118]:
import os
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

zipf = zipfile.ZipFile('heatmaps.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('heatmaps/', zipf)
zipf.close()

In [120]:
list(filter(lambda x: x=="<start>",inputs))

['<start>']